In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
from matplotlib.patches import Polygon, Rectangle
matplotlib.use('Qt5Agg')
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from casadi import *
import calendar
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
import casadi as cd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
def custom_date_parser(date_string):
    return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M:%S')

# Specify the path to the main directory containing folders and files
path = 'D:\\mlinternship\\iitgdata'
folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
df_list = []

# Iterate through each folder
for folder in folders:
    # Construct the full path to the current folder
    folder_path = os.path.join(path, folder)
    # Iterate through files in the current folder
    for filename in os.listdir(folder_path):
        # Check if the file has the '.xlsx' extension
        if filename.endswith('.xlsx'):
            # Construct the full path to the Excel file
            file_path = os.path.join(folder_path, filename)

            # Use the custom date parser function
            df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)

            # Append the dataframe to the list
            df_list.append(df)


bill_path = 'D:\\mlinternship\\IITGuwahatiElectricityBills'
for filename in os.listdir(bill_path):
    # Check if the file has the '.xlsx' extension
    if filename.endswith('.xlsx'):
        file_path = os.path.join(bill_path, filename)
        # Use the custom date parser function
        bill_df = pd.read_excel(file_path)

bill_df['Month'] = pd.to_datetime(bill_df['Month'])
#I assumed its in kilowatts
bill_df['MW'] = bill_df['Number of units of electricity consumed']/1000
bill_df.drop(['Number of units of electricity consumed'], axis=1)
bill_df

C:\Users\jaina\AppData\Local\Temp\ipykernel_20044\1131752821.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
C:\Users\jaina\AppData\Local\Temp\ipykernel_20044\1131752821.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
C:\Users\jaina\AppData\Local\Temp\ipykernel_20044\1131752821.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_pars

,Month,MW
0,2022-01-01,1415.4246
1,2022-02-01,1353.4260
2,2022-03-01,2254.4145
3,2022-04-01,2269.1010
4,2022-05-01,2690.4150
5,2022-06-01,2708.4910
6,2022-07-01,3113.6625
7,2022-08-01,3606.0495
8,2022-09-01,3335.3190
9,2022-10-01,2787.5835


,Month,Number of units of electricity consumed,MW
0,2022-01-01,1415424.6,1415.4246
1,2022-02-01,1353426.0,1353.4260
2,2022-03-01,2254414.5,2254.4145
3,2022-04-01,2269101.0,2269.1010
4,2022-05-01,2690415.0,2690.4150
5,2022-06-01,2708491.0,2708.4910
6,2022-07-01,3113662.5,3113.6625
7,2022-08-01,3606049.5,3606.0495
8,2022-09-01,3335319.0,3335.3190
9,2022-10-01,2787583.5,2787.5835


In [3]:
#read the power data
power_df = pd.concat(df_list, ignore_index=True)
power_df.to_csv('power_datacsv.csv')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df['Time'] = power_df['Time'].round('min')
#replace all the 'NR' values in MW column to NaN
power_df['MW'] = power_df['MW'].replace('NR', np.nan)
power_df['MW'] = power_df['MW'].replace('nr', np.nan)
power_df = power_df[['Time', 'MW']]
power_df['MW'] = power_df['MW'].astype(str)
power_df['MW'] = pd.to_numeric(power_df['MW'].str.replace(',', '.'), errors='coerce')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df = power_df.sort_values('Time')
full_power_df = power_df.copy()

# read the temperature data csv
temperature_data_csv_path = 'D:\\mlinternship\\iitgdata\\temperaturedata'
filename = 'guwahati_temperature_data.csv'
file = os.path.join(temperature_data_csv_path, filename)
temperature_df = pd.read_csv(file)
temperature_df.rename(columns={'valid': 'Time'}, inplace = True)
temperature_df = temperature_df.rename(columns={'tmpc': 'temperature'})
temperature_df = temperature_df[['Time', 'temperature']]
temperature_df['Time'] = pd.to_datetime(temperature_df['Time'])
temperature_df['Time'] = pd.DatetimeIndex(temperature_df['Time']) + timedelta(hours=5,minutes=30)
temperature_df['temperature'] = pd.to_numeric(temperature_df['temperature'], errors='coerce')
temperature_df.set_index('Time', inplace=True)
temperature_df['temperature'] = temperature_df['temperature'].interpolate(method='polynomial', order = 5)
temperature_df.reset_index(inplace=True)

# joining the two dataframes such that the temperature data is only taken if there exists a reading in the power data dataframe
df = pd.merge(power_df, temperature_df, on='Time', how='left')
df['temperature'] = df['temperature'].interpolate(method='polynomial', order = 5)
df['Time'] = pd.to_datetime(df['Time'])

full_model_start_time = pd.Timestamp('2022-02-07 00:00:00')
full_model_end_time = pd.Timestamp('2022-12-25 23:00:00')
df = df[(df['Time'] >= full_model_start_time) & (df['Time'] <= full_model_end_time)]
df = df.drop(df[df['MW'] > 20].index)
df = df.sort_values('Time')
df.reset_index(drop=True)
df

,Time,MW,temperature
0,2022-02-07 00:00:00,2.18,12.000000
1,2022-02-07 01:00:00,2.05,11.633781
2,2022-02-07 02:00:00,NaN,11.703556
3,2022-02-07 03:00:00,1.71,12.000000
4,2022-02-07 04:00:00,NaN,12.000000
...,...,...,...
5753,2022-12-25 18:00:00,NaN,19.000000
5754,2022-12-25 19:00:00,2.60,18.000000
5755,2022-12-25 20:00:00,NaN,18.000000
5756,2022-12-25 21:00:00,2.60,18.000000


,Time,MW,temperature
160,2022-02-07 00:00:00,2.18,12.000000
161,2022-02-07 01:00:00,2.05,11.633781
162,2022-02-07 02:00:00,NaN,11.703556
163,2022-02-07 03:00:00,1.71,12.000000
164,2022-02-07 04:00:00,NaN,12.000000
...,...,...,...
5914,2022-12-25 18:00:00,NaN,19.000000
5915,2022-12-25 19:00:00,2.60,18.000000
5916,2022-12-25 20:00:00,NaN,18.000000
5917,2022-12-25 21:00:00,2.60,18.000000


In [4]:
TcoolStPt = 31
CDH = df['temperature'] - TcoolStPt
CDH.clip(lower=0, inplace=True)
CDH = pd.DataFrame(data=CDH.values, columns=['CDH'], index=df.index)
# Concatenate CDH with the original DataFrame using the index
df = pd.concat([df, CDH], axis=1)
df = df.sort_values('Time')
df.reset_index(inplace=True, drop = True)

numOmegas = 24 * 7
num_of_rows = df.shape[0]
omegas = np.zeros((num_of_rows, numOmegas))  # Assuming numOmegas columns for omegas
concatenated_data = np.concatenate((df, omegas), axis=1)
column_names = ['Time', 'MW', 'temperature', 'CDH']
for i in range(1, numOmegas + 1,1):
    column_names.append('omega' + str(i))

df = pd.DataFrame(concatenated_data, columns=column_names)
df['Time'] = pd.to_datetime(df['Time'])
for i in range(0,num_of_rows):
        datetime = df.Time.loc[i]
        hourOfWeekIndex = int(datetime.dayofweek*24+(datetime.hour+1))
        x = np.zeros((1,numOmegas))
        x[0,hourOfWeekIndex-1]=1
        omegas[i,:]=x

df.iloc[:,4:]=omegas
DF = df.copy()
df = df.dropna()
df.reset_index(inplace=True, drop = True)
df

,Time,MW,temperature,CDH,omega1,omega2,omega3,omega4,omega5,omega6,...,omega159,omega160,omega161,omega162,omega163,omega164,omega165,omega166,omega167,omega168
0,2022-02-07 00:00:00,2.18,12.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-02-07 01:00:00,2.05,11.633781,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-02-07 03:00:00,1.71,12.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-02-07 05:00:00,1.7,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-02-07 14:00:00,2.5,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,2022-12-25 15:00:00,2.4,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3814,2022-12-25 17:00:00,2.6,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3815,2022-12-25 19:00:00,2.6,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3816,2022-12-25 21:00:00,2.6,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
start_time = pd.Timestamp('2022-07-04 00:00:00')
end_time = pd.Timestamp('2022-08-28 23:00:00')
training_mask = (df['Time'] >= start_time) & (df['Time'] <= end_time) & (df['CDH'] > 0)
training_df = df[training_mask]

In [6]:
training_df

,Time,MW,temperature,CDH,omega1,omega2,omega3,omega4,omega5,omega6,...,omega159,omega160,omega161,omega162,omega163,omega164,omega165,omega166,omega167,omega168
1689,2022-07-04 14:00:00,5.0,34.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1690,2022-07-04 15:00:00,5.0,34.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1691,2022-07-04 16:00:00,5.0,34.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1692,2022-07-04 17:00:00,5.0,34.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1701,2022-07-05 07:00:00,4.3,32.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2557,2022-08-27 15:00:00,4.47,32.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2558,2022-08-27 16:00:00,4.89,32.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2559,2022-08-27 17:00:00,5.42,32.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2577,2022-08-28 14:00:00,4.0,32.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


SVM Part

In [7]:

x = training_df.loc[:, 'temperature':'omega168']
x = x.to_numpy()
y = training_df['MW']
y = y.to_numpy()
y = y.reshape(-1,1)
scaler = StandardScaler()
x = scaler.fit_transform(x)
y = scaler.fit_transform(y)

In [8]:
# C is inversely proportional to the number of misclassifications you allow
# epsilon is the allowed error in each instance

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
y = y.reshape(len(y),)

In [10]:
svr_model = SVR()
svr_model.fit(x,y)

SVR()

In [11]:
print("score of the model: ", svr_model.score(X_test,y_test))
y_pred = svr_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on Test Set:", mse)

score of the model:  0.6033733532718922
Mean Squared Error on Test Set: 0.5850803312179021


In [12]:
'''yhat = svr_model.predict(x)
plt.ion()
plt.figure(figsize=(12, 6))
plt.plot(training_df['Time'], yhat, marker='o', linestyle='-', color = 'g')
plt.plot(training_df['Time'], y, marker='x', linestyle='-', color = 'r')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()'''

"yhat = svr_model.predict(x)\nplt.ion()\nplt.figure(figsize=(12, 6))\nplt.plot(training_df['Time'], yhat, marker='o', linestyle='-', color = 'g')\nplt.plot(training_df['Time'], y, marker='x', linestyle='-', color = 'r')\nplt.xlabel('Time')\nplt.ylabel('Power Consumption Changes')\n\nplt.grid(True)\nplt.legend()\nplt.show()"

In [13]:
param_grid = {'C':[0.01, 0.1,0.5,1,1.5,2.5,5,10,15],
             'kernel':['linear','rbf', 'poly'],
              'gamma':['scale','auto'],
              'degree':[1,2,3,4,5,6],
              'epsilon':[0.01, 0.1, 0.25]}

svr_grid = SVR()
# Create a GridSearchCV object
grid_search = GridSearchCV(svr_grid,param_grid=param_grid)

# Fit the grid search to the data
grid_search.fit(x, y)

# Get the best parameters and the best model
best_params = grid_search.best_params_

# Make predictions on the test set
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("Best Parameters:", best_params)
print("Mean Squared Error on Test Set:", mse)
print("score of the model: ", grid_search.score(X_test,y_test))

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.01, 0.1, 0.5, 1, 1.5, 2.5, 5, 10, 15],
                         'degree': [1, 2, 3, 4, 5, 6],
                         'epsilon': [0.01, 0.1, 0.25],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']})

Best Parameters: {'C': 1.5, 'degree': 1, 'epsilon': 0.25, 'gamma': 'scale', 'kernel': 'poly'}
Mean Squared Error on Test Set: 0.5289808374900885
score of the model:  0.6414032662482703


In [14]:
yhat = grid_search.predict(x)
plt.ion()


plt.figure(figsize=(12, 6))
plt.plot(training_df['Time'], yhat, marker='o', linestyle='-', color = 'g', label = 'predicted')
plt.plot(training_df['Time'], y, marker='x', linestyle='-', color = 'r', label = 'actual')
plt.xlabel('Time')
plt.ylabel('Power Consumption Changes')

plt.grid(True)
plt.legend()
plt.show()

<Figure size 1200x600 with 0 Axes>

Text(0.5, 0, 'Time')

Text(0, 0.5, 'Power Consumption Changes')